In [ ]:
import numpy as np
from matplotlib import pyplot as plt

#Define the amount to win. This is the same as the number of states
trials = 10

#Define tolerance
theta = 1e-20


state =  list(range(trials+1))

initial_action = [0]*(win + 1)
for i in states[1:win]:
    initial_action[i] = 1